In [1]:
#importing the libraries
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as tfs
from numpy import dot
from numpy.linalg import norm
import math
from torch.utils.data import TensorDataset, DataLoader
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, Dense, concatenate, AveragePooling2D, UpSampling2D
from keras import Model
import math

# train в dataset 

In [2]:
#setting the path to the directory containing the pics
path = 'C:\\Users\\79778\\SimpleCube++\\TRATEST\\TrainPng\\trainPNG'

training_data = []
for img in os.listdir(path):
    pic = cv2.imread(os.path.join(path,img), cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(648,432))
    training_data.append([pic])
np.save(os.path.join(path,'features'),np.array(training_data))


In [3]:
saved = np.load(os.path.join(path,'features.npy'))
print(saved.shape)
saved = saved.reshape(1772,432, 648, 3)

(1772, 1, 432, 648, 3)


In [4]:
saved1=saved[0:500]


In [5]:
saved1 = np.where(saved1 == [0,0,0], [2048,2048,2048], saved1)
#при вычитании теневого тока из черного прямоугольника числа в uint перепрыгнут через 0, так что сначала
#прибавим этот самый ток 

In [6]:
saved2=saved[500:1000]
saved2 = np.where(saved2 == [0,0,0], [2048,2048,2048], saved2)

In [7]:
saved3=saved[1000:1500]
saved3 = np.where(saved3 == [0,0,0], [2048,2048,2048], saved3)

In [8]:
saved4=saved[1500:]
saved4 = np.where(saved4 == [0,0,0], [2048,2048,2048], saved4)

In [9]:
reworked_saved=np.concatenate((saved1,saved2), axis=0)


In [10]:
reworked_saved=np.concatenate((reworked_saved,saved3), axis=0 )

In [11]:
reworked_saved=np.concatenate((reworked_saved,saved4), axis=0)

In [12]:
reworked_saved.shape

(1772, 432, 648, 3)

In [13]:

train_dataloader_features = reworked_saved-2048# вычитание теневого тока
train_dataloader_colors = pd.read_csv ('C:\\Users\\79778\\SimpleCube++\\SimpleCube++\\train\\gt.csv')
train_dataloader_colors = train_dataloader_colors.to_numpy()
train_dataloader_colors = np.delete(train_dataloader_colors,0,1)#оставляем только ответы без индексов 
train_dataloader_colors = train_dataloader_colors.astype('float32')

# test в dataset

In [14]:
path = 'C:\\Users\\79778\\SimpleCube++\\TRATEST\\TestPng\\testPNG'

#appending the pics to the training data list
training_data = []
for img in os.listdir(path):
    pic = cv2.imread(os.path.join(path,img), cv2.IMREAD_ANYCOLOR |cv2.IMREAD_ANYDEPTH)
    pic = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)
    pic = cv2.resize(pic,(648,432))
    training_data.append([pic])

np.save(os.path.join(path,'features'),np.array(training_data))

In [15]:
saved = np.load(os.path.join(path,'features.npy'))
saved = saved.reshape(462, 432, 648, 3)

In [16]:
saved = np.where(saved == [0,0,0], [2048,2048,2048], saved)

In [17]:
test_dataloader_features = saved-2048
test_dataloader_colors = pd.read_csv ('C:\\Users\\79778\\SimpleCube++\\SimpleCube++\\test\\gt.csv')
test_dataloader_colors = test_dataloader_colors.to_numpy()
test_dataloader_colors = np.delete(test_dataloader_colors,0,1)  
test_dataloader_colors = test_dataloader_colors.astype('float32')

# Grey-World

In [42]:
answers = np.sum(train_dataloader_features,axis=(1,2))
answers = np.array(answers)
angle_cos =[]
for i in range(1772):
        angle_cos.append( dot (answers[i], train_dataloader_colors[i])/(norm( train_dataloader_colors[i])*norm(answers[i])))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

3.093686721120101

In [102]:
tr = np.sum(angle-np.average(angle))**2# пригшодится для отклонения

1.7894562888014682

In [57]:
answers = np.average(test_dataloader_features,(1,2))
answers = np.array(answers)
angle =[]
for i in range(462):
        angle.append( dot (answers[i], test_dataloader_colors[i])/(norm( test_dataloader_colors[i])*norm(answers[i])))
angle = np.array(angle)
angle = np.arccos(angle)*180/np.pi
np.average(angle)

3.4985113726620964

In [100]:
np.sqrt(np.sum((tr + np.sum((angle-np.average(angle))**2)))/(1772+461))# стандартное отклонение

1.5940744156073838

# MAX-RGB

In [105]:
lkdsjf = np.amax(train_dataloader_features, axis=(2,1))
angle_cos =[]
for i in range(1772):
        angle_cos.append( dot (lkdsjf[i], train_dataloader_colors[i])/(norm( train_dataloader_colors[i])*norm(lkdsjf[i])))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

5.581871450437838

In [106]:
tr = np.sum(angle-np.average(angle))**2

In [107]:
testi =  np.amax(test_dataloader_features, axis=(2,1))
angleB =[]
for i in range(462):
        angleB.append( dot (testi[i], test_dataloader_colors[i])/(norm( test_dataloader_colors[i])*norm(testi[i])))
angleB = np.array(angleB)
angleB = np.arccos(angleB)*180/np.pi
np.average(angleB)

5.988402216193486

In [108]:
np.sqrt(np.sum((tr + np.sum((angleB-np.average(angleB))**2)))/(1772+461))

2.895840461711341

# Shades of Grey

In [109]:
train_dataloader_features = train_dataloader_features.astype('float64')# для чтобы степени влезли в диапазон

train_dataloader_features_fornorm = train_dataloader_features
norma =np.amax(train_dataloader_features_fornorm, axis=3, keepdims=True)
train_dataloader_features_normed = train_dataloader_features/norma
train_dataloader_features_normed = np.nan_to_num(train_dataloader_features_normed, posinf=0, neginf=0)

In [110]:
n=2
prpose = np.power(np.sum(np.power(train_dataloader_features,n),axis=(1,2)),(1/n))
print(prpose)
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(1772):
        angle_cos.append( np.clip(dot (answers[i], train_dataloader_colors[i])/(norm( train_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

[[ 358005.53730913  727917.41899683  405964.30272131]
 [ 409834.40901296  928537.23809118  620069.42408895]
 [ 947394.54130262 1932661.38061482 1241245.42689389]
 ...
 [ 743830.96857351  740650.36694651  156292.05343523]
 [ 730092.71425416  723734.86133666  119878.81179341]
 [ 689453.05546643  694391.33123837  109078.82778523]]


2.933959593918391

In [111]:
tr = np.sum(angle-np.average(angle))**2

# test


In [112]:
test_dataloader_features = test_dataloader_features.astype('float64')

test_dataloader_features_fornorm = test_dataloader_features
norma =np.amax(test_dataloader_features_fornorm, axis=3, keepdims=True)
test_dataloader_features_normed = test_dataloader_features/norma
test_dataloader_features_normed = np.nan_to_num(test_dataloader_features_normed, posinf=0, neginf=0)

In [113]:
n=2
prpose = np.power(np.sum(np.power(test_dataloader_features,n),axis=(1,2)),(1/n))
print(np.isnan(prpose).sum())
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(462):
        angle_cos.append(np.clip( dot (answers[i], test_dataloader_colors[i])/(norm( test_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

0


3.3356660130356843

In [114]:
np.sqrt(np.sum((tr + np.sum((angle-np.average(angle))**2)))/(1772+461))

1.670520336927425

# General Grey-World

In [115]:
sigma = 1
train_dataloader_features_blured = []
for i in range(1772):
    train_dataloader_features_blured.append(cv2.GaussianBlur(train_dataloader_features[i],(3,3),sigma))
train_dataloader_features_blured = np.array(train_dataloader_features_blured)

In [117]:
n=2
prpose = np.power(np.sum(np.power(train_dataloader_features_blured,n),axis=(1,2)),(1/n))
print(prpose)
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(1772):
        angle_cos.append( np.clip(dot (answers[i], train_dataloader_colors[i])/(norm( train_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

[[ 355001.03822997  721800.06871683  401607.33831749]
 [ 397844.20669729  899632.36421362  598014.09723417]
 [ 924747.5063466  1884309.14318525 1206368.08886181]
 ...
 [ 742589.97258902  739507.97144693  155998.3463174 ]
 [ 729735.62503444  723435.08930625  119720.07092078]
 [ 686885.11585858  691976.33089487  108544.36995686]]


2.94644039511389

In [118]:
tr = np.sum(angle-np.average(angle))**2

In [119]:
sigma = 1
test_dataloader_features_blured = []
for i in range(462):
    test_dataloader_features_blured.append(cv2.GaussianBlur(test_dataloader_features[i],(3,3),sigma))

test_dataloader_features_blured = np.array(test_dataloader_features_blured)

n=2
prpose = np.power(np.sum(np.power(test_dataloader_features_blured,n),axis=(1,2)),(1/n))
print(prpose)
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(462):
        angle_cos.append( np.clip(dot (answers[i], test_dataloader_colors[i])/(norm( test_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

[[ 795719.60586223  938597.6810903   306074.05393315]
 [1628115.5203109  3195839.0077214  2877769.29552145]
 [ 546355.63737927 1146781.94197917  702386.2448328 ]
 ...
 [1065374.73374758  993379.50856225  150726.78471436]
 [1164239.02937461 1063801.3681758   135663.01078191]
 [ 961137.46166625  944628.47058162  178788.29609459]]


3.3473674375358704

1.6751001584314833

# Grey Edge

In [128]:
train_dataloader_features_diff_2 = []#тут сразу код для второй производной, (подредаченная первая)
for i in range (1772):
    train_dataloader_features_blured=cv2.GaussianBlur(train_dataloader_features_diff[i],(3,3),1)
    x = cv2.Sobel(train_dataloader_features_blured,cv2.CV_64F,1,0)  
    y = cv2.Sobel(train_dataloader_features_blured,cv2.CV_64F,0,1) 
    train_dataloader_features_diff_2.append(cv2.addWeighted(x,1,y,1,0)) 
train_dataloader_features_diff_2=np.array(train_dataloader_features_diff_2).astype("float64")

In [129]:
n=2
prpose = np.power(np.sum(np.power(np.abs(train_dataloader_features_diff_2),n),axis=(1,2)),(1/n))
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(1772):
        angle_cos.append( np.clip(dot (answers[i], train_dataloader_colors[i])/(norm( train_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

3.011704951362594

In [130]:
tr = np.sum(angle-np.average(angle))**2

In [131]:
test_dataloader_features_diff_2 = []
for i in range (462):
    test_dataloader_features_blured=cv2.GaussianBlur(test_dataloader_features_diff[i],(3,3),1)
    x = cv2.Sobel(test_dataloader_features_blured,cv2.CV_64F,1,0)  
    y = cv2.Sobel(test_dataloader_features_blured,cv2.CV_64F,0,1) 
    test_dataloader_features_diff_2.append(cv2.addWeighted(x,1,y,1,0)) 
    

test_dataloader_features_diff_2=np.array(test_dataloader_features_diff_2).astype("float64")

In [132]:
n=2
prpose = np.power(np.sum(np.power(np.abs(test_dataloader_features_diff_2),n),axis=(1,2)),(1/n))
prpose = np.array(prpose)
answers = prpose
angle_cos =[]
for i in range(462):
        angle_cos.append( np.clip(dot (answers[i], test_dataloader_colors[i])/(norm( test_dataloader_colors[i])*norm(answers[i])),-1,1))
angle = np.arccos(angle_cos)*180/np.pi
np.average(angle)

3.416054501372754

In [133]:
np.sqrt(np.sum((tr + np.sum((angle-np.average(angle))**2)))/(1772+461))

1.6843980030109347

## Цербер

In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_dataloader_features.astype('float16'),train_dataloader_colors))
train_dataset = train_dataset.shuffle(buffer_size=256).batch(1)

In [25]:
train_dataloader_features.shape

(1772, 432, 648, 3)

In [19]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_dataloader_features.astype('float16'),test_dataloader_colors))
test_dataset = test_dataset.shuffle(buffer_size=256).batch(1)

In [20]:
pi = tf.constant(math.pi)
def maxim(y_true, y_pred):  
    return tf.math.reduce_max(tf.clip_by_value(tf.compat.v1.losses.cosine_distance(y_true, y_pred, axis=1),-1,1))
def disspersion(y_true, y_pred):
    return tf.math.sqrt(tf.math.reduce_sum((y_pred-y_true)**2/(tf.cast(tf.shape(y_true),tf.float32 ))))

In [21]:
inp = Input(shape=(432, 648, 3))
lay = AveragePooling2D(pool_size=(432, 648),input_shape = (432, 648, 3), padding='same')(inp)
lay = UpSampling2D((432, 648),input_shape = (432, 648, 3))(lay)
out = tf.keras.layers.concatenate([lay,inp], axis=-1)
conv_model = Model(inp, out)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./65535),
    #tf.keras.layers.RandomCrop(100,100),
    tf.keras.layers.Resizing(432, 648),
    
    #tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="constant"),
    
    tf.keras.layers.RandomFlip("horizontal_and_vertical",),
    tf.keras.layers.RandomZoom(.7, .7),
    tf.keras.layers.RandomRotation(1),
    conv_model,
    tf.keras.layers.Conv2D(16, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(32, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(64, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(128, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(256, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(1024, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(1500, 3,padding='same', activation='relu',input_shape=(64,64, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(2400, 3,padding='same', activation='relu',input_shape=(64,64, 6)),
    tf.keras.layers.MaxPooling2D(input_shape=(432, 648, 6)),
    tf.keras.layers.Conv2D(16, 3,padding='same', activation='relu',input_shape=(432, 648, 6)),
    #,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4800,activation='relu'),
    tf.keras.layers.Dense(1000,activation='relu'),
    tf.keras.layers.Dense(500,activation='relu'),
    tf.keras.layers.Dense(400,activation='relu'),
    tf.keras.layers.Dense(3)
])


In [27]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=1e-5),
  loss='mae',
  metrics=[tf.keras.metrics.CosineSimilarity(axis=1),disspersion],
  
)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=0.0001)
history = model.fit(
    train_dataset,
    
    validation_data=test_dataset,
    epochs=20,
)

Epoch 1/20
  22/1772 [..............................] - ETA: 2:23 - loss: 0.3310 - cosine_similarity: 0.9449 - disspersion: 89.6454

KeyboardInterrupt: 

In [26]:
model.save("cosine_0,9994",  save_format='h5')# тут я сохранил данные свокй лучше модели

# FC4

In [28]:
from keras.models import Model
from keras.layers import Input, Activation, Concatenate
from keras.layers import Flatten, Dropout, Dense
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D


def SqueezeNet(inputs=(3, 224, 224)):
    """
    Arguments:
    inputs -- shape of the input images (cols, rows, channel)
    """

    input_img = Input(shape=(432, 648, 3))
    rescale = tf.keras.layers.Rescaling(1./65535)(input_img)
    #tf.keras.layers.RandomCrop(100,100),
    resize = tf.keras.layers.Resizing(224, 224)(rescale)
    
    #tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="constant"),
    
    flip = tf.keras.layers.RandomFlip("horizontal_and_vertical",)(resize)
    
    rotate = tf.keras.layers.RandomRotation(1)(flip)
    conv1 = Convolution2D(
        96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
        strides=(2, 2), padding='same', name='conv1',
        )(rotate)
    maxpool1 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool1',
        )(conv1)
    fire2_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze',
        )(maxpool1)
    fire2_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1',
        )(fire2_squeeze)
    fire2_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2',
        )(fire2_squeeze)
    merge2 = Concatenate(axis=-1)([fire2_expand1, fire2_expand2])

    fire3_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_squeeze',
        )(merge2)
    fire3_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand1',
        )(fire3_squeeze)
    fire3_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand2',
        )(fire3_squeeze)
    merge3 = Concatenate(axis=-1)([fire3_expand1, fire3_expand2])

    fire4_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_squeeze',
        )(merge3)
    fire4_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand1',
        )(fire4_squeeze)
    fire4_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand2',
        )(fire4_squeeze)
    merge4 = Concatenate(axis=-1)([fire4_expand1, fire4_expand2])
    maxpool4 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool4',
        )(merge4)

    fire5_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_squeeze',
        )(maxpool4)
    fire5_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand1',
        )(fire5_squeeze)
    fire5_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand2',
        )(fire5_squeeze)
    merge5 = Concatenate(axis=-1)([fire5_expand1, fire5_expand2])

    fire6_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_squeeze',
        )(merge5)
    fire6_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand1',
        )(fire6_squeeze)
    fire6_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand2',
        )(fire6_squeeze)
    merge6 = Concatenate(axis=-1)([fire6_expand1, fire6_expand2])

    fire7_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_squeeze',
        )(merge6)
    fire7_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand1',
        )(fire7_squeeze)
    fire7_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand2',
        )(fire7_squeeze)
    merge7 = Concatenate(axis=-1)([fire7_expand1, fire7_expand2])

    fire8_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_squeeze',
        )(merge7)
    fire8_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand1',
        )(fire8_squeeze)
    fire8_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand2',
        )(fire8_squeeze)
    merge8 = Concatenate(axis=-1)([fire8_expand1, fire8_expand2])

    maxpool8 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool8',
        )(merge8)
    fire9_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_squeeze',
        )(maxpool8)
    fire9_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand1',
        )(fire9_squeeze)
    fire9_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand2',
        )(fire9_squeeze)
    merge9 = Concatenate(axis=-1)([fire9_expand1, fire9_expand2])

    #fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
    conv10 = Convolution2D(
        4, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='valid', name='conv10',
        )(merge9)

    global_avgpool10 = GlobalAveragePooling2D()(conv10)
    flatten = Flatten()(global_avgpool10)
    #dense = Dense(400)(flatten)
    #dense = Activation('relu')(dense)
    dense = Dense(3)(flatten)
    return Model(inputs=input_img, outputs=dense)
    

In [29]:
from keras.models import Model
model = SqueezeNet()

In [26]:
def disspersion(y_true, y_pred):
    delta = tf.math.acos(tf.clip_by_value(tf.math.reduce_sum(tf.math.multiply(y_true ,y_pred))/(tf.norm(y_true)*tf.norm(y_true)),-1,1))*180/3.14159265359
    squared_difference = tf.square(delta)
    return tf.sqrt(tf.reduce_mean(squared_difference))

In [33]:
import keras.backend as K
class ReduceLR(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=[]):
        if K.eval(self.model.optimizer.iterations) >= 50:
            K.set_value(self.model.optimizer.lr, 1e-4)
reduce_lr = ReduceLR()

In [34]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=1e-7),
  loss='mae',
  metrics=[tf.keras.metrics.CosineSimilarity(axis=1), disspersion],
  
)
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=0.0001)
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,
    callbacks=[reduce_lr]
)

Epoch 1/100
1772/1772 [==============================] - 41s 22ms/step - loss: 0.0551 - cosine_similarity: 0.9800 - disspersion: 20.4436 - val_loss: 0.0264 - val_cosine_similarity: 0.9896 - val_disspersion: 14.7739
Epoch 2/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0340 - cosine_similarity: 0.9820 - disspersion: 10.2279 - val_loss: 0.0322 - val_cosine_similarity: 0.9893 - val_disspersion: 19.9329 loss: 0.0335 - cosine_sim
Epoch 3/100
1772/1772 [==============================] - 38s 22ms/step - loss: 0.0336 - cosine_similarity: 0.9819 - disspersion: 10.1409 - val_loss: 0.0255 - val_cosine_similarity: 0.9896 - val_disspersion: 3.8366
Epoch 4/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0329 - cosine_similarity: 0.9819 - disspersion: 9.8646 - val_loss: 0.0242 - val_cosine_similarity: 0.9896 - val_disspersion: 5.9696
Epoch 5/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0328 - cosine_similarity: 0.9819 - d

1772/1772 [==============================] - 40s 22ms/step - loss: 0.0123 - cosine_similarity: 0.9983 - disspersion: 4.1527 - val_loss: 0.0135 - val_cosine_similarity: 0.9980 - val_disspersion: 7.4154 0.0124 - cosine_similarity: 0.9983 - dissp - ETA: 1s - loss: 0.0124 - cosine_similarity: 0.9983
Epoch 38/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0116 - cosine_similarity: 0.9984 - disspersion: 3.9983 - val_loss: 0.0137 - val_cosine_similarity: 0.9975 - val_disspersion: 3.6570
Epoch 39/100
1772/1772 [==============================] - 40s 22ms/step - loss: 0.0111 - cosine_similarity: 0.9985 - disspersion: 3.9077 - val_loss: 0.0127 - val_cosine_similarity: 0.9980 - val_disspersion: 5.1224
Epoch 40/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0113 - cosine_similarity: 0.9985 - disspersion: 3.7833 - val_loss: 0.0150 - val_cosine_similarity: 0.9970 - val_disspersion: 3.6730A: 0s - loss: 0.0113 - cosine_similarity: 0.9985 - disspersion

1772/1772 [==============================] - 38s 22ms/step - loss: 0.0101 - cosine_similarity: 0.9988 - disspersion: 3.5212 - val_loss: 0.0115 - val_cosine_similarity: 0.9982 - val_disspersion: 4.0632: 0.0102 - cosine_similar
Epoch 75/100
1772/1772 [==============================] - 39s 22ms/step - loss: 0.0105 - cosine_similarity: 0.9986 - disspersion: 3.6922 - val_loss: 0.0115 - val_cosine_similarity: 0.9984 - val_disspersion: 1.7480
Epoch 76/100
1772/1772 [==============================] - 38s 21ms/step - loss: 0.0103 - cosine_similarity: 0.9988 - disspersion: 3.8011 - val_loss: 0.0117 - val_cosine_similarity: 0.9982 - val_disspersion: 3.2724
Epoch 77/100
1772/1772 [==============================] - 37s 21ms/step - loss: 0.0103 - cosine_similarity: 0.9987 - disspersion: 3.4922 - val_loss: 0.0118 - val_cosine_similarity: 0.9981 - val_disspersion: 2.6182
Epoch 78/100
1772/1772 [==============================] - 38s 21ms/step - loss: 0.0104 - cosine_similarity: 0.9987 - disspersion: 3.

In [35]:
model.save("fc4",  save_format='h5')

In [38]:
from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"disspersion": disspersion})

In [39]:
model = keras.models.load_model('./fc4')